In [ ]:
import random, math, re, tqdm, multiprocessing
from concurrent.futures import as_completed, ThreadPoolExecutor

In [ ]:
EXPLORATION_CONSTANT = 2
NUM_WEEKS = 18
TIMEOUT = 180000
THRESHOLD = 0.4

In [3]:
class GameNode:
    def __init__(self, teams, week, value, winner, previous_winners=None, previous_losers=None):
        self.teams = teams
        self.week = week
        self.value = value
        self.tree_value = self.value
        self.winner = winner
        self.loser = (self.teams[0] if winner != self.teams[0] else self.teams[1])
        self.id = self.winner + "_" + self.loser
        self.next = []
        self.previous_winners = [winner for winner in previous_winners] if previous_winners != None else []
        self.previous_losers = [loser for loser in previous_losers] if previous_losers != None else []
        self.sims = 1
    
    def copy(self):
        return GameNode(self.teams, self.week, self.value, self.winner, self.previous_winners, self.previous_losers)
    
    def __repr__(self):
        return "{}. {}".format(self.week, self.id)
    
    def pretty_print(self):
        print({
            "id": self.id,
            "teams": self.teams,
            "week": self. week,
            "value": self.value,
            "tree_value": self.tree_value,
            "next": self.next,
            "previous_winners": self.previous_winners,
            "previous_losers": self.previous_losers,
            "sims": self.sims,
            "winner": self.winner,
        })
        

In [4]:
class Season:
    def __init__(self, filename=None, threshold=0):
        self.games = [{} for i in range(NUM_WEEKS + 1)]
        self.threshold = threshold
        if filename != None:
            self.create_from_file(filename)
    
    def add_game(self, teams, week, value, winner):
        if value >= self.threshold:
            game = GameNode(teams, week, value, winner)
            self.games[week][game.id] = game

    def get_week(self, week):
        return self.games[week]
    
    def get_eligible_games(self, week, previous_winners, previous_losers):
        if week == NUM_WEEKS:
            return []
        
        eligible = []
        for game_id in self.games[week]:
            game = self.games[week][game_id]
            if game.winner not in previous_winners and game.loser not in previous_losers:
                eligible.append(game.copy())

        return eligible
    
    def create_from_file(self, filename):
        matchup_regex = re.compile("([A-Y]+)-logo\s+[A-Y4][9a-y].+\s(\d+)%")

        current_week = 0
        current_game_teams = []
        current_game_values = []

        with open(filename) as f:
            for line in f.readlines():
                if re.match("Week \d{1,2}", line) != None:
                    current_week += 1
                match = matchup_regex.match(line)
                if match != None:
                    team = match.group(1)
                    odds = round(float(match.group(2)) / 100, 2)
                    current_game_teams.append(team)
                    current_game_values.append(odds)
                    if len(current_game_teams) == 2:
                        self.add_game(current_game_teams, current_week, current_game_values[0], current_game_teams[0])
                        self.add_game(current_game_teams, current_week, current_game_values[1], current_game_teams[1])
                        current_game_teams = []
                        current_game_values = []
    
    def pretty_print(self):
        for week in range(1, NUM_WEEKS + 1):
            print("WEEK {}".format(week))
            for game in self.games[week]:
                self.games[week][game].pretty_print()
            print()

In [18]:
class Tree:
    def __init__(self, season):
        self.season = season
        self.nodes = []
              
    def __get_selection_val(self, current, next_node):
        return (next_node.tree_value / (next_node.sims)) + math.sqrt(2 * math.log(current.sims) / (next_node.sims))
    
    def select(self, current):
        if current.next == None or len(current.next) == 0:
            return None
        best_node = current.next[0]
        best_val = 0
        for node in current.next:
            val = self.__get_selection_val(current, node)
            if val > best_val:
                best_val = val
                best_node = node
        
        return best_node
    
    def simulate(self, current):
        previous_winners = current.previous_winners if current.winner == "" else current.previous_winners + [current.winner]
        previous_losers = current.previous_losers if current.loser == "" else current.previous_losers + [current.loser]
        eligible_next_nodes = current.next if current.next != None and len(current.next) > 0 else self.season.get_eligible_games(current.week + 1, previous_winners, previous_losers)
        next_node = random.choice(eligible_next_nodes).copy() if len(eligible_next_nodes) > 0 else None
        if next_node != None:
            next_node.previous_winners = previous_winners
            next_node.previous_losers = previous_losers
        return next_node
    
    def expand(self, current):
        previous_winners = current.previous_winners if current.winner == "" else current.previous_winners + [current.winner]
        previous_losers = current.previous_losers if current.loser == "" else current.previous_losers + [current.loser]
        eligible_next_nodes = self.season.get_eligible_games(current.week + 1, previous_winners, previous_losers)
        current.next = eligible_next_nodes
        for node in current.next:
            node.previous_winners = previous_winners
            node.previous_losers = previous_losers
        return self.simulate(current)

                    
        

In [21]:
class RolloutManager:
    def __init__(self, root, tree):
        self.root = root
        self.tree = tree
    
    def __forward(self):
        nodes = [self.root]
        should_select = True
        week = self.root.week
        
        running_sum = 0
        current = self.root
        next_node = current
        while week <= NUM_WEEKS and next_node != None:
            if should_select:
                if len(current.next) > 0:
                    next_node = self.tree.select(current)
                else:
                    next_node = self.tree.expand(current)
                    should_select = False
                if next_node != None:
                    nodes.append(next_node)
                    running_sum += next_node.value
            else:
                next_node = self.tree.simulate(current)
                if next_node != None:
                    running_sum += next_node.value
            current = next_node
            week += 1
        return nodes, running_sum
        
    def __backward(self, nodes, running_sum):
        for node in nodes:
            node.tree_value = node.tree_value + (running_sum / (NUM_WEEKS - 1))
            node.sims = node.sims + 1
    
    def rollout(self, count=1, print_frequency=10_000):
        for i in range(count):
            nodes, running_sum = self.__forward()
            self.__backward(nodes, running_sum)
            if i % print_frequency == 0 and i > 0:
                print(i)
    
    def choose(self):
        selections = []
        total = 0
        depth = 0
        current = self.root

        while current != None:
            next_node = None
            options = current.next
            if len(options) > 0:
                depth += 1
                best_val = 0
                for option in options:
                    if (option.tree_value / option.sims if option.sims > 0 else 0) > best_val:
                        best_val = option.tree_value / option.sims
                        next_node = option
            else:
                next_node = self.tree.simulate(current)
            current = next_node
            if next_node != None:
                selections.append(next_node)
                total += next_node.value
        return selections, total, depth
    
    def multi_choose(self, count=1):
        best_score = 0
        best_games = []
        best_depth = 0
        for i in range(count):
            games, score, depth = self.choose()
            if score > best_score:
                best_games = games
                best_score = score
                best_depth = depth
        return best_games, best_score, best_depth
    
    def pretty_print_selections(self):
        games, score, depth = self.choose()
        print("Depth Calculated: {}".format(depth))
        print("Total Score: {}".format(score))
        for game in games:
            print(game.week, game.id)
        print("\n")

In [20]:
rollout_manager = RolloutManager(GameNode(["", ""], 0, 0, ""), Tree(Season("2022_matchups_raw", threshold=THRESHOLD)))
rollout_manager.rollout(1000)
output_games, _, _ = rollout_manager.multi_choose()
for game in output_games:
    print(game.week, game.id)

1 CIN_PIT
2 LAR_ATL
3 MIN_DET
4 DEN_OAK
5 PHI_ARI
6 MIA_MIN
7 TEN_IND
8 BUF_GB
9 NO_BAL
10 NYG_HOU
11 ATL_CHI
12 OAK_SEA
13 IND_DAL
14 KC_DEN
15 CHI_PHI
16 GB_MIA
17 DAL_TEN


In [22]:
NUM_ROLLOUTS = 100_000
NUM_SIMULATIONS = 100_000

def calculate(game, num_rollouts, num_simulations):
    rollout_manager = RolloutManager(game, Tree(Season("2022_matchups_raw", threshold=THRESHOLD)))
    rollout_manager.rollout(num_rollouts)
    print("Finished rollout for", game)
    return rollout_manager.multi_choose(num_simulations)

season = Season("2022_matchups_raw", threshold=THRESHOLD)
opening_games = [season.games[1][game] for game in filter(lambda x: x != "LAR_BUF" and x != "BUF_LAR" and season.games[1][x].value > THRESHOLD, season.games[1])]
futures = []
results = []

with ThreadPoolExecutor(max_workers=len(opening_games)) as executor:
    for game in opening_games: 
        futures.append(executor.submit(calculate, game, NUM_ROLLOUTS, NUM_SIMULATIONS))
    
    best_overall_games = []
    best_overall_score = 0
    for i, future in enumerate(futures):
        try:
            start_game = opening_games[i]
            best_games, best_score, best_depth = future.result(timeout=TIMEOUT)
            results.append((best_games, best_score, best_depth))
            if best_score + start_game.value > best_overall_score:
                best_overall_score = best_score + start_game.value
                best_overall_games = [start_game] + best_games
        except:
            print("{} timed out".format(game))
    
    print("Best Score:", best_overall_score)
    for game in best_overall_games:
        print(game.week, game.id)


10000
10000
10000
10000
10000
10000
1000010000

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
20000
30000
30000
3000030000

30000
30000
3000030000

30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
40000
4000040000

40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
40000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
5000050000

50000
50000
50000
50000
50000
50000
6000060000

60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
60000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
70000
8000080000

80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
80000
8000

In [23]:
results

[([2. GB_CHI,
   3. MIN_DET,
   4. DAL_WSH,
   5. TB_ATL,
   6. IND_JAX,
   7. SF_KC,
   8. NO_OAK,
   9. CIN_CAR,
   10. LAR_ARI,
   11. BUF_CLE,
   12. MIA_HOU,
   13. DEN_BAL,
   14. ARI_NE,
   15. WSH_NYG,
   16. KC_SEA,
   17. PHI_NO],
  10.99,
  5),
 ([2. LAR_ATL,
   3. MIN_DET,
   4. PHI_JAX,
   5. BUF_PIT,
   6. GB_NYJ,
   7. BAL_CLE,
   8. DAL_CHI,
   9. TB_LAR,
   10. SF_LAC,
   11. DEN_OAK,
   12. MIA_HOU,
   13. NYG_WSH,
   14. ARI_NE,
   15. LAC_TEN,
   16. KC_SEA,
   17. NE_MIA],
  11.149999999999999,
  5),
 ([2. GB_CHI,
   3. MIN_DET,
   4. PHI_JAX,
   5. TB_ATL,
   6. LAR_CAR,
   7. DEN_NYJ,
   8. NO_OAK,
   9. ARI_SEA,
   10. BUF_MIN,
   11. CIN_PIT,
   12. TEN_CIN,
   13. WSH_NYG,
   14. DAL_HOU,
   15. LAC_TEN,
   16. SF_WSH,
   17. KC_DEN],
  11.02,
  5),
 ([2. LAR_ATL,
   3. LAC_JAX,
   4. DAL_WSH,
   5. BUF_PIT,
   6. GB_NYJ,
   7. BAL_CLE,
   8. NO_OAK,
   9. CIN_CAR,
   10. SF_LAC,
   11. NYG_DET,
   12. KC_LAR,
   13. TB_NO,
   14. ARI_NE,
   15. DEN_ARI,
   16